# Raw Data Exploration

**This notebook should be run FIRST** before any preprocessing scripts.

## Objectives
1. Understand the structure of raw data files
2. Explore data coverage (countries, years, variables)
3. Identify data quality issues
4. Understand what preprocessing will be needed
5. Get familiar with the data before running the pipeline

## Raw Data Sources
- **FAO Food Balance Sheets**: Nutrition consumption data
- **FAO Population Data**: Demographic data
- **WHO Obesity Data**: Health outcome data

---


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up paths
PROJECT_ROOT = Path().resolve().parent
RAW_DATA_DIR = PROJECT_ROOT / "data" / "raw"

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print(f"📁 Raw data directory: {RAW_DATA_DIR}")
print(f"   Exists: {RAW_DATA_DIR.exists()}")


## 1. FAO Food Balance Sheet Data

This is the main nutrition dataset containing food consumption, nutrients, and supply data.


In [ ]:
# Load FAO Food Balance Sheet data
fbs_file = RAW_DATA_DIR / "FoodBalanceSheet_data" / "FoodBalanceSheets_E_All_Data_(Normalized).csv"

if fbs_file.exists():
    print(f"📂 Loading: {fbs_file.name}")
    fbs_df = pd.read_csv(fbs_file, low_memory=False)
    print(f"✅ Loaded {len(fbs_df):,} rows, {len(fbs_df.columns)} columns")
    
    print("\n📋 Column Names:")
    print(list(fbs_df.columns))
    
    print("\n📊 Dataset Info:")
    print(f"   Shape: {fbs_df.shape}")
    print(f"   Memory usage: {fbs_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print("\n🔍 First few rows:")
    display(fbs_df.head())
    
    print("\n📈 Data Types:")
    print(fbs_df.dtypes)
    
else:
    print(f"❌ File not found: {fbs_file}")
    fbs_df = None


In [ ]:
# Explore FAO data structure
if fbs_df is not None:
    print("=" * 70)
    print("FAO FOOD BALANCE SHEET - DATA EXPLORATION")
    print("=" * 70)
    
    # Clean column names for exploration
    fbs_df.columns = fbs_df.columns.str.strip()
    
    # Basic statistics
    print("\n📊 Coverage Statistics:")
    if 'Area' in fbs_df.columns:
        print(f"   Countries/Areas: {fbs_df['Area'].nunique()}")
        print(f"   Sample countries: {', '.join(fbs_df['Area'].unique()[:10])}")
    
    if 'Year' in fbs_df.columns:
        print(f"   Year range: {fbs_df['Year'].min()} - {fbs_df['Year'].max()}")
        print(f"   Unique years: {fbs_df['Year'].nunique()}")
    
    if 'Item' in fbs_df.columns:
        print(f"   Food items: {fbs_df['Item'].nunique()}")
        print(f"   Sample items: {', '.join(fbs_df['Item'].unique()[:10])}")
    
    if 'Element' in fbs_df.columns:
        print(f"   Elements (variables): {fbs_df['Element'].nunique()}")
        print(f"\n   Available elements:")
        for elem in sorted(fbs_df['Element'].unique()):
            count = len(fbs_df[fbs_df['Element'] == elem])
            print(f"     - {elem}: {count:,} rows")
    
    # Check for missing values
    print("\n🔍 Missing Values:")
    missing = fbs_df.isnull().sum()
    missing_pct = (missing / len(fbs_df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_pct
    })
    missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
    if len(missing_df) > 0:
        display(missing_df.head(10))
    else:
        print("   ✅ No missing values!")


In [ ]:
# Visualize FAO data coverage
if fbs_df is not None and 'Year' in fbs_df.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Year distribution
    if 'Year' in fbs_df.columns:
        year_counts = fbs_df['Year'].value_counts().sort_index()
        axes[0, 0].bar(year_counts.index, year_counts.values)
        axes[0, 0].set_title('Data Coverage by Year')
        axes[0, 0].set_xlabel('Year')
        axes[0, 0].set_ylabel('Number of Records')
        axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Top countries by record count
    if 'Area' in fbs_df.columns:
        country_counts = fbs_df['Area'].value_counts().head(20)
        axes[0, 1].barh(range(len(country_counts)), country_counts.values)
        axes[0, 1].set_yticks(range(len(country_counts)))
        axes[0, 1].set_yticklabels(country_counts.index)
        axes[0, 1].set_title('Top 20 Countries by Record Count')
        axes[0, 1].set_xlabel('Number of Records')
    
    # Element distribution
    if 'Element' in fbs_df.columns:
        element_counts = fbs_df['Element'].value_counts().head(15)
        axes[1, 0].barh(range(len(element_counts)), element_counts.values)
        axes[1, 0].set_yticks(range(len(element_counts)))
        axes[1, 0].set_yticklabels(element_counts.index, fontsize=8)
        axes[1, 0].set_title('Top 15 Elements by Record Count')
        axes[1, 0].set_xlabel('Number of Records')
    
    # Item distribution (top items)
    if 'Item' in fbs_df.columns:
        item_counts = fbs_df['Item'].value_counts().head(15)
        axes[1, 1].barh(range(len(item_counts)), item_counts.values)
        axes[1, 1].set_yticks(range(len(item_counts)))
        axes[1, 1].set_yticklabels(item_counts.index, fontsize=8)
        axes[1, 1].set_title('Top 15 Food Items by Record Count')
        axes[1, 1].set_xlabel('Number of Records')
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Coverage visualizations created!")


## 2. FAO Population Data

Population data for calculating per-capita consumption rates.


In [ ]:
# Load FAO Population data
pop_file = RAW_DATA_DIR / "Population_data" / "Population_E_All_Area_Groups_NOFLAG.csv"

if pop_file.exists():
    print(f"📂 Loading: {pop_file.name}")
    pop_df = pd.read_csv(pop_file, low_memory=False)
    print(f"✅ Loaded {len(pop_df):,} rows, {len(pop_df.columns)} columns")
    
    print("\n📋 Column Names:")
    print(list(pop_df.columns))
    
    print("\n🔍 First few rows:")
    display(pop_df.head())
    
    # Check for year columns (wide format)
    year_cols = [col for col in pop_df.columns if col.startswith('Y')]
    print(f"\n📅 Year columns found: {len(year_cols)}")
    if year_cols:
        print(f"   Year range: {year_cols[0]} to {year_cols[-1]}")
        print(f"   Sample years: {year_cols[:5]} ... {year_cols[-5:]}")
    
    # Check Element column
    if 'Element' in pop_df.columns:
        print(f"\n📊 Elements (population types):")
        print(pop_df['Element'].unique())
    
else:
    print(f"❌ File not found: {pop_file}")
    pop_df = None


In [ ]:
# Explore Population data structure
if pop_df is not None:
    print("=" * 70)
    print("FAO POPULATION DATA - DATA EXPLORATION")
    print("=" * 70)
    
    # Clean column names
    pop_df.columns = pop_df.columns.str.strip()
    
    print("\n📊 Coverage Statistics:")
    if 'Area' in pop_df.columns:
        print(f"   Countries/Areas: {pop_df['Area'].nunique()}")
        print(f"   Sample countries: {', '.join(pop_df['Area'].unique()[:10])}")
    
    if 'Element' in pop_df.columns:
        print(f"\n   Population types:")
        for elem in pop_df['Element'].unique():
            count = len(pop_df[pop_df['Element'] == elem])
            print(f"     - {elem}: {count} rows")
    
    # Check year columns
    year_cols = [col for col in pop_df.columns if col.startswith('Y')]
    if year_cols:
        print(f"\n   Year columns: {len(year_cols)}")
        print(f"   Year range: {year_cols[0]} ({year_cols[0].replace('Y', '')}) to {year_cols[-1]} ({year_cols[-1].replace('Y', '')})")
        
        # Check for missing values in year columns
        print(f"\n   Missing values in year columns:")
        missing_years = pop_df[year_cols].isnull().sum()
        if missing_years.sum() > 0:
            print(f"     Total missing: {missing_years.sum():,}")
            print(f"     Columns with most missing: {missing_years.nlargest(5).to_dict()}")
        else:
            print("     ✅ No missing values in year columns")
    
    print("\n⚠️  Note: Population data is in WIDE format (years as columns)")
    print("   It will be transformed to LONG format during preprocessing.")


## 3. WHO Obesity Data

Health outcome data - obesity prevalence by country and year.


In [ ]:
# Load WHO Obesity data
obesity_file = RAW_DATA_DIR / "data.csv"

if obesity_file.exists():
    print(f"📂 Loading: {obesity_file.name}")
    obesity_df = pd.read_csv(obesity_file, low_memory=False)
    print(f"✅ Loaded {len(obesity_df):,} rows, {len(obesity_df.columns)} columns")
    
    print("\n📋 Column Names:")
    print(list(obesity_df.columns))
    
    print("\n🔍 First few rows:")
    display(obesity_df.head())
    
    print("\n📈 Data Types:")
    print(obesity_df.dtypes)
    
else:
    print(f"❌ File not found: {obesity_file}")
    obesity_df = None


In [ ]:
# Explore Obesity data structure
if obesity_df is not None:
    print("=" * 70)
    print("WHO OBESITY DATA - DATA EXPLORATION")
    print("=" * 70)
    
    # Clean column names
    obesity_df.columns = obesity_df.columns.str.strip()
    
    print("\n📊 Coverage Statistics:")
    
    # Check for location/country column
    location_cols = [col for col in obesity_df.columns if any(x in col.lower() for x in ['country', 'location', 'area', 'region'])]
    if location_cols:
        loc_col = location_cols[0]
        print(f"   Countries/Locations: {obesity_df[loc_col].nunique()}")
        print(f"   Sample countries: {', '.join(str(x) for x in obesity_df[loc_col].unique()[:10])}")
    
    # Check for year column
    year_cols = [col for col in obesity_df.columns if 'year' in col.lower()]
    if year_cols:
        year_col = year_cols[0]
        print(f"   Year range: {obesity_df[year_col].min()} - {obesity_df[year_col].max()}")
        print(f"   Unique years: {obesity_df[year_col].nunique()}")
    
    # Check for indicator column
    indicator_cols = [col for col in obesity_df.columns if 'indicator' in col.lower()]
    if indicator_cols:
        ind_col = indicator_cols[0]
        print(f"\n   Indicators (health metrics):")
        indicators = obesity_df[ind_col].unique()
        for ind in indicators[:10]:
            count = len(obesity_df[obesity_df[ind_col] == ind])
            print(f"     - {ind}: {count} rows")
        if len(indicators) > 10:
            print(f"     ... and {len(indicators) - 10} more")
    
    # Check for dimension columns (sex, age groups, etc.)
    dim_cols = [col for col in obesity_df.columns if 'dim' in col.lower()]
    if dim_cols:
        print(f"\n   Dimensions:")
        for dim_col in dim_cols:
            print(f"     {dim_col}: {obesity_df[dim_col].unique()}")
    
    # Check for value column
    value_cols = [col for col in obesity_df.columns if 'value' in col.lower() or 'obesity' in col.lower()]
    if value_cols:
        val_col = value_cols[0]
        print(f"\n   Value column: {val_col}")
        # Convert to numeric before formatting
        numeric_values = pd.to_numeric(obesity_df[val_col], errors='coerce')
        if not numeric_values.isna().all():
            print(f"   Value range: {numeric_values.min():.2f} - {numeric_values.max():.2f}")
        else:
            print(f"   Value range: {obesity_df[val_col].min()} - {obesity_df[val_col].max()} (non-numeric)")
        print(f"   Missing values: {obesity_df[val_col].isnull().sum()} ({obesity_df[val_col].isnull().sum()/len(obesity_df)*100:.1f}%)")
    
    # Check for missing values
    print("\n🔍 Missing Values by Column:")
    missing = obesity_df.isnull().sum()
    missing_pct = (missing / len(obesity_df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_pct
    })
    missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
    if len(missing_df) > 0:
        display(missing_df)
    else:
        print("   ✅ No missing values!")

In [ ]:
# Visualize Obesity data coverage
if obesity_df is not None:
    # Find relevant columns
    location_cols = [col for col in obesity_df.columns if any(x in col.lower() for x in ['country', 'location', 'area'])]
    year_cols = [col for col in obesity_df.columns if 'year' in col.lower()]
    value_cols = [col for col in obesity_df.columns if 'value' in col.lower() or 'obesity' in col.lower()]
    
    if location_cols and year_cols:
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Year distribution
        if year_cols:
            year_col = year_cols[0]
            year_counts = obesity_df[year_col].value_counts().sort_index()
            axes[0].bar(year_counts.index, year_counts.values)
            axes[0].set_title('Data Coverage by Year')
            axes[0].set_xlabel('Year')
            axes[0].set_ylabel('Number of Records')
            axes[0].tick_params(axis='x', rotation=45)
        
        # Top countries by record count
        if location_cols:
            loc_col = location_cols[0]
            country_counts = obesity_df[loc_col].value_counts().head(20)
            axes[1].barh(range(len(country_counts)), country_counts.values)
            axes[1].set_yticks(range(len(country_counts)))
            axes[1].set_yticklabels(country_counts.index)
            axes[1].set_title('Top 20 Countries by Record Count')
            axes[1].set_xlabel('Number of Records')
        
        plt.tight_layout()
        plt.show()
        
        print("📊 Coverage visualizations created!")


## 4. Data Quality Assessment

Identify potential issues that need to be addressed during preprocessing.


In [ ]:
print("=" * 70)
print("DATA QUALITY ASSESSMENT")
print("=" * 70)

quality_issues = []

# Check FAO data
if fbs_df is not None:
    print("\n📋 FAO Food Balance Sheet:")
    
    # Check for missing values
    if fbs_df.isnull().sum().sum() > 0:
        total_missing = fbs_df.isnull().sum().sum()
        quality_issues.append(f"FAO: {total_missing:,} missing values")
        print(f"   ⚠️  Missing values: {total_missing:,}")
    else:
        print("   ✅ No missing values")
    
    # Check for duplicate rows
    if fbs_df.duplicated().sum() > 0:
        quality_issues.append(f"FAO: {fbs_df.duplicated().sum():,} duplicate rows")
        print(f"   ⚠️  Duplicate rows: {fbs_df.duplicated().sum():,}")
    else:
        print("   ✅ No duplicate rows")
    
    # Check data types
    if 'Year' in fbs_df.columns:
        if fbs_df['Year'].dtype == 'object':
            quality_issues.append("FAO: Year column is not numeric")
            print("   ⚠️  Year column needs conversion to numeric")
        else:
            print("   ✅ Year column is numeric")
    
    # Check for inconsistent country names
    if 'Area' in fbs_df.columns:
        areas_with_spaces = fbs_df['Area'].str.contains('  ', na=False).sum()
        if areas_with_spaces > 0:
            quality_issues.append(f"FAO: {areas_with_spaces} country names with extra spaces")
            print(f"   ⚠️  Country names with extra spaces: {areas_with_spaces}")
        else:
            print("   ✅ Country names look clean")

# Check Population data
if pop_df is not None:
    print("\n📋 FAO Population:")
    
    # Check format
    year_cols = [col for col in pop_df.columns if col.startswith('Y')]
    if year_cols:
        print(f"   ℹ️  Data is in WIDE format ({len(year_cols)} year columns)")
        print("   ⚠️  Needs transformation to LONG format during preprocessing")
        quality_issues.append("Population: Needs wide-to-long transformation")
    
    # Check for missing values
    if pop_df.isnull().sum().sum() > 0:
        total_missing = pop_df.isnull().sum().sum()
        quality_issues.append(f"Population: {total_missing:,} missing values")
        print(f"   ⚠️  Missing values: {total_missing:,}")
    else:
        print("   ✅ No missing values")

# Check Obesity data
if obesity_df is not None:
    print("\n📋 WHO Obesity:")
    
    # Check for missing values
    if obesity_df.isnull().sum().sum() > 0:
        total_missing = obesity_df.isnull().sum().sum()
        quality_issues.append(f"Obesity: {total_missing:,} missing values")
        print(f"   ⚠️  Missing values: {total_missing:,}")
    else:
        print("   ✅ No missing values")
    
    # Check for duplicate rows
    if obesity_df.duplicated().sum() > 0:
        quality_issues.append(f"Obesity: {obesity_df.duplicated().sum():,} duplicate rows")
        print(f"   ⚠️  Duplicate rows: {obesity_df.duplicated().sum():,}")
    else:
        print("   ✅ No duplicate rows")

# Summary
print("\n" + "=" * 70)
print("SUMMARY OF QUALITY ISSUES")
print("=" * 70)
if quality_issues:
    print(f"\n⚠️  Found {len(quality_issues)} potential issues:")
    for i, issue in enumerate(quality_issues, 1):
        print(f"   {i}. {issue}")
    print("\n💡 These will be addressed during preprocessing.")
else:
    print("\n✅ No major quality issues detected!")


## 5. Data Coverage Comparison

Compare coverage across datasets to understand what will be available in the final merged dataset.


In [ ]:
print("=" * 70)
print("DATA COVERAGE COMPARISON")
print("=" * 70)

coverage_summary = []

# FAO coverage
if fbs_df is not None and 'Area' in fbs_df.columns and 'Year' in fbs_df.columns:
    fbs_countries = set(fbs_df['Area'].unique())
    fbs_years = set(fbs_df['Year'].unique())
    coverage_summary.append({
        'Dataset': 'FAO Food Balance Sheet',
        'Countries': len(fbs_countries),
        'Years': len(fbs_years),
        'Year Range': f"{min(fbs_years)}-{max(fbs_years)}"
    })

# Population coverage
if pop_df is not None and 'Area' in pop_df.columns:
    pop_countries = set(pop_df['Area'].unique())
    year_cols = [col for col in pop_df.columns if col.startswith('Y')]
    if year_cols:
        pop_years = set([int(col.replace('Y', '')) for col in year_cols])
        coverage_summary.append({
            'Dataset': 'FAO Population',
            'Countries': len(pop_countries),
            'Years': len(pop_years),
            'Year Range': f"{min(pop_years)}-{max(pop_years)}"
        })

# Obesity coverage
if obesity_df is not None:
    location_cols = [col for col in obesity_df.columns if any(x in col.lower() for x in ['country', 'location', 'area'])]
    year_cols = [col for col in obesity_df.columns if 'year' in col.lower()]
    if location_cols and year_cols:
        loc_col = location_cols[0]
        year_col = year_cols[0]
        ob_countries = set(obesity_df[loc_col].dropna().unique())
        ob_years = set(obesity_df[year_col].dropna().unique())
        coverage_summary.append({
            'Dataset': 'WHO Obesity',
            'Countries': len(ob_countries),
            'Years': len(ob_years),
            'Year Range': f"{int(min(ob_years))}-{int(max(ob_years))}"
        })

# Display summary
if coverage_summary:
    coverage_df = pd.DataFrame(coverage_summary)
    display(coverage_df)
    
    # Estimate final panel coverage
    if len(coverage_summary) >= 2:
        print("\n📊 Estimated Final Panel Coverage:")
        print("   (Based on intersection of all datasets)")
        
        # Find common countries (if possible)
        if fbs_df is not None and pop_df is not None:
            common_countries = fbs_countries.intersection(pop_countries)
            if obesity_df is not None and location_cols:
                loc_col = location_cols[0]
                ob_countries = set(obesity_df[loc_col].dropna().unique())
                common_countries = common_countries.intersection(ob_countries)
            print(f"   Estimated countries: ~{len(common_countries)} (intersection)")
        
        # Find common years
        if fbs_df is not None and pop_df is not None:
            common_years = fbs_years.intersection(pop_years)
            if obesity_df is not None and year_cols:
                year_col = year_cols[0]
                ob_years = set(obesity_df[year_col].dropna().unique())
                common_years = common_years.intersection(ob_years)
            print(f"   Estimated years: ~{len(common_years)} (intersection)")
            if common_years:
                print(f"   Year range: {min(common_years)}-{max(common_years)}")
        
        print("\n💡 Note: Actual coverage may vary after data cleaning and filtering.")
else:
    print("⚠️  Could not compare coverage - check data loading above.")
